# Classification

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Tutorial 3

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$
$\DeclareMathOperator*{\argmin}{arg\,min}$

Setting up the environment

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as opt

%matplotlib inline

## The data set

We will predict the incidence of diabetes based on various measurements (see [description](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes)). Instead of directly using the raw data, we use a normalised version, where the label to be predicted (the incidence of diabetes) is in the first column. Download the data from [mldata.org](http://mldata.org/repository/data/download/csv/diabetes_scale/).

Read in the data using pandas.

In [6]:
data = pd.read_csv('diabetes_scale.csv', names=['class', 'pregnant', 'plasGluc', 'dBP', 'Triceps', 'insulin', 'BMI', 'age', 'diabFunc'])
data

,class,pregnant,plasGluc,dBP,Triceps,insulin,BMI,age,diabFunc
0,-1,-0.294118,0.487437,0.180328,-0.292929,-1.000000,0.001490,-0.531170,-0.033333
1,1,-0.882353,-0.145729,0.081967,-0.414141,-1.000000,-0.207153,-0.766866,-0.666667
2,-1,-0.058824,0.839196,0.049180,-1.000000,-1.000000,-0.305514,-0.492741,-0.633333
3,1,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,-1.000000
4,-1,-1.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000
5,1,-0.411765,0.165829,0.213115,-1.000000,-1.000000,-0.236960,-0.894962,-0.700000
6,-1,-0.647059,-0.216080,-0.180328,-0.353535,-0.791962,-0.076006,-0.854825,-0.833333
7,1,0.176471,0.155779,-1.000000,-1.000000,-1.000000,0.052161,-0.952178,-0.733333
8,-1,-0.764706,0.979899,0.147541,-0.090909,0.283688,-0.090909,-0.931682,0.066667
9,-1,-0.058824,0.256281,0.573770,-1.000000,-1.000000,-1.000000,-0.868488,0.100000


## Classification via Logistic Regression

Implement binary classification using logistic regression for a data set with two classes. Make sure you use appropriate [python style](https://www.python.org/dev/peps/pep-0008/) and [docstrings](https://www.python.org/dev/peps/pep-0257/).

Use ```scipy.optimize.fmin_bfgs``` to optimise your cost function. ```fmin_bfgs``` requires the cost function to be optimised, and the gradient of this cost function. Implement these two functions as ```cost``` and ```grad``` by following the equations in the lectures.

Implement the function ```train``` that takes a matrix of examples, and a vector of labels, and returns the maximum likelihood weight vector for logistic regresssion. Also implement a function ```test``` that takes this maximum likelihood weight vector and the a matrix of examples, and returns the predictions. See the section **Putting everything together** below for expected usage.

We add an extra column of ones to represent the constant basis.

In [7]:
data['ones'] = np.ones((data.shape[0], 1)) # add a column of ones
data.head()

,class,pregnant,plasGluc,dBP,Triceps,insulin,BMI,age,diabFunc,ones
0,-1,-0.294118,0.487437,0.180328,-0.292929,-1.000000,0.001490,-0.531170,-0.033333,1
1,1,-0.882353,-0.145729,0.081967,-0.414141,-1.000000,-0.207153,-0.766866,-0.666667,1
2,-1,-0.058824,0.839196,0.049180,-1.000000,-1.000000,-0.305514,-0.492741,-0.633333,1
3,1,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,-1.000000,1
4,-1,-1.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,1


In [8]:
cost = 
initial_guess = 
grad_cost = 
args


SyntaxError: invalid syntax (<ipython-input-8-ae0d7471e652>, line 1)

## Performance measure

There are many ways to compute the [performance of a binary classifier](http://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers). The key concept is the idea of a confusion matrix or contingency table:

|              |    | Label |    |
|:-------------|:--:|:-----:|:--:|
|              |    |  +1   | -1 |
|**Prediction**| +1 |    TP | FP |
|              | -1 |    FN | TN |

where
* TP - true positive
* FP - false positive
* FN - false negative
* TN - true negative

Implement three functions, the first one which returns the confusion matrix for comparing two lists (one set of predictions, and one set of labels). Then implement two functions that take the confusion matrix as input and returns the **accuracy** and **balanced accuracy** respectively. The [balanced accuracy](http://en.wikipedia.org/wiki/Accuracy_and_precision) is the average accuracy of each class.


In [ ]:
# Solution goes here

## Putting everything together

Consider the following code, which trains on all the examples, and predicts on the training set. Discuss the results.

In [ ]:
y = data['diabetes']
X = data[['num preg', 'plasma', 'bp', 'skin fold', 'insulin', 'bmi', 'pedigree', 'age', 'ones']]
theta_best = train(X, y)
pred = predict(theta_best, X)
cmatrix = confusion_matrix(pred, y)
[accuracy(cmatrix), balanced_accuracy(cmatrix)]

### Solution description

## Fisher's discriminant

In the lectures, you saw that the Fisher criterion
$$
J(w) = \frac{w^T S_B w}{w^T S_W w}
$$
is maximum for Fisher's linear discriminant.

Define $S_B$ and $S_W$ as in the lectures and prove this result.

### Solution description

### Solution description

## (optional) Effect of regularization parameter

By splitting the data into two halves, train on one half and report performance on the second half. By repeating this experiment for different values of the regularization parameter $\lambda$ we can get a feeling about the variability in the performance of the classifier due to regularization. Plot the values of accuracy and balanced accuracy for at least 3 different choices of $\lambda$. Note that you may have to update your implementation of logistic regression to include the regularisation parameter.


In [ ]:
### Solution

def split_data(data):
    """Randomly split data into two equal groups"""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(N/2)]
    test_idx = idx[int(N/2):]

    X_train = data.loc[train_idx].drop('diabetes', axis=1)
    t_train = data.loc[train_idx]['diabetes']
    X_test = data.loc[test_idx].drop('diabetes', axis=1)
    t_test = data.loc[test_idx]['diabetes']
    
    return X_train, t_train, X_test, t_test


### Solution description